# QR Algorithm

Ci sono delle proprietà:

- Dato che gli eigenvalue di una matrice tirangolare sono gli elementi sulla diagonale, questo algoritmo permette di trovarli.
- Similarity transformation preserva gli eigenvalue.

La matrice A viene **decomposta** in A = QR dove Q è ortogonale e R è triangolare superiore.

## Gram-Schmidt
Come prima implementazione proviamo con l'algoritmo di **Gram-Schmidt**


In [16]:
import numpy as np
from itertools import product
from math import copysign, hypot



In [12]:
def qr_decomp_gram_schmidt(A):
    m, n = A.shape
    rank = np.linalg.matrix_rank(A)

    if rank < n:
        print(f"Il rango della matrice è {rank} il quale è inferiore del numero delle colonne {n}!")
    
    Q = np.zeros((m, n))
    
    for i, column in enumerate(A.T):
        Q[:,i] = column

        for prev in Q.T[:i]:
            Q[:,i] -= (prev @ column) / (prev @ prev) * prev
    
    Q /= np.linalg.norm(Q,axis=0)
    R = Q.T @ A

    return Q, R

In [13]:
# Tests

#TODO: rendere i test non dipendenti da una sola funzione

def test_orthonormality(A):
    Q, _ = qr_decomp_gram_schmidt(A)
    I = Q.T @ Q

    assert len(set(I.shape)) == 1, "L'identità computata deve essere una matrice quadrata!"
    assert np.allclose(I, np.eye(I.shape[0])), "L'identità  computata deve essere una matrice identità!"

def test_upper_triangular(A):
    _, R = qr_decomp_gram_schmidt(A)

    assert len(set(R.shape)) == 1, "La matrice triangolare superiore computata dovrebbe essere una matrice quadrata!"

    for i, row in enumerate(R):
        assert np.allclose(row[:i], 0), f"La riga {i} della matrice triangolare superiore calcolata non è giusta!"

def test_multiplication(A):
    Q,R = qr_decomp_gram_schmidt(A)
    
    assert np.allclose(Q @ R, A), "La moltiplicazione fra le matrici ottenute non restituisce la matrice originaria!"

In [14]:
test_matricies = (
    np.random.random((2, 2)),
    np.random.random((5, 2)),
    np.random.random((5, 5)),
    np.random.random((9, 8)),
)

tests = (
    test_orthonormality,
    test_upper_triangular,
    test_multiplication,
)

for test, matrix in product(tests, test_matricies):
    test(matrix)

print("tutti i test sono superati")

tutti i test sono superati


# Givens Rotation
proviamo ad implementare le givens rotation per migliorare la velocità di esecuzione.

In [22]:
def qr_decomp_givens_rotation(A):

    (num_rows, num_cols) = np.shape(A)
    if num_rows == num_cols: 
        # Initialize Q,R
        # Q = orthogonal matrix
        # R =  upper triangular matrix
        Q = np.identity(num_rows)
        R = np.copy(A)

        # Iterate over lower triangular matrix.
        (rows, cols) = np.tril_indices(num_rows, -1, num_cols)
        #print(rows, cols)
        for (row, col) in zip(rows, cols):

            # Compute Givens rotation matrix and
            # zero-out lower triangular matrix entries.
            if R[row, col] != 0:
                (c, s) = Givens_Rotation_Matrix_Entries(R[col, col], R[row, col])

                G = np.identity(num_rows)
                G[[col, row], [col, row]] = c
                G[row, col] = s
                G[col, row] = -s

                R = np.dot(G, R)
                Q = np.dot(Q, G.T)
                #R = G @ R
                #Q = Q @ G.T
        return Q, R
    else:
        Q = np.eye(num_rows)
        R = np.copy(A)

        rows, cols = np.tril_indices(num_rows, -1, num_cols)
        for (row, col) in zip(rows, cols):
            # If the subdiagonal element is nonzero, then compute the nonzero 
            # components of the rotation matrix
            if R[row, col] != 0:
                r = np.sqrt(R[col, col]**2 + R[row, col]**2)
                c, s = R[col, col]/r, -R[row, col]/r

                # The rotation matrix is highly discharged, so it makes no sense 
                # to calculate the total matrix product
                R[col], R[row] = R[col]*c + R[row]*(-s), R[col]*s + R[row]*c
                Q[:, col], Q[:, row] = Q[:, col]*c + Q[:, row]*(-s), Q[:, col]*s + Q[:, row]*c

        return Q[:, :num_cols], R[:num_cols]



##### Compute matrix entries for Givens rotation. #####

def Givens_Rotation_Matrix_Entries(a, b):
    r = hypot(a, b)
    c = a/r
    s = -b/r

    return (c, s)

In [23]:
def test_orthonormality(A):
    Q, _ = qr_decomp_givens_rotation(A)
    I = Q.T @ Q

    assert len(set(I.shape)) == 1, "L'identità computata deve essere una matrice quadrata!"
    assert np.allclose(I, np.eye(I.shape[0])), "L'identità  computata deve essere una matrice identità!"

def test_upper_triangular(A):
    _, R = qr_decomp_givens_rotation(A)

    assert len(set(R.shape)) == 1, "La matrice triangolare superiore computata dovrebbe essere una matrice quadrata!"

    for i, row in enumerate(R):
        assert np.allclose(row[:i], 0), f"La riga {i} della matrice triangolare superiore calcolata non è giusta!"

def test_multiplication(A):
    Q, R = qr_decomp_givens_rotation(A)
    
    assert np.allclose(Q @ R, A), "La moltiplicazione fra le matrici ottenute non restituisce la matrice originaria!"

In [29]:
test_matricies = (
    np.random.random((3, 3)),
    np.random.random((4, 5)),
    np.random.random((5, 5)),
    np.random.random((9, 8)),
)

tests = (
    test_orthonormality,
    test_multiplication,
)

for test, matrix in product(tests, test_matricies):
    test(matrix)

print("tutti i test sono superati")

tutti i test sono superati


In [27]:
A = np.random.random((4, 4))

Q, R = qr_decomp_givens_rotation(A)

print(A)
print(Q)
print(R)

print(Q @ R)

[[0.35017775 0.93848075 0.16708714 0.02033739]
 [0.91653041 0.38546321 0.71378994 0.29711919]
 [0.56154242 0.9004278  0.09630576 0.11294642]
 [0.18937849 0.19613426 0.14601127 0.42272211]]
[[ 0.30550353  0.71297018  0.60257606 -0.18773172]
 [ 0.79960326 -0.54617603  0.19061298 -0.16122363]
 [ 0.48990317  0.4391397  -0.75115862  0.05396237]
 [ 0.16521836  0.02283828  0.19060299  0.96739434]]
[[ 1.14623146e+00  1.06845423e+00  6.93098715e-01  3.68964881e-01]
 [ 1.65483215e-17  8.58470982e-01 -2.25100477e-01 -8.85259237e-02]
 [ 6.63161982e-18 -2.39492889e-17  1.92229625e-01  6.46210192e-02]
 [ 2.37833226e-17  6.07842044e-18  2.70775298e-18  3.63313227e-01]]
[[0.35017775 0.93848075 0.16708714 0.02033739]
 [0.91653041 0.38546321 0.71378994 0.29711919]
 [0.56154242 0.9004278  0.09630576 0.11294642]
 [0.18937849 0.19613426 0.14601127 0.42272211]]
